In [1]:
from tensorflow.keras.utils import Sequence
import os
import argparse
import random
import math
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from phos_label_generator import gen_label
from phoc_label_generator import gen_phoc_label

# Setting random seeds

random.seed(73)
np.random.seed(73) 

def getphoclabel(x):
    return all_phoc_labels[x]

def getphoslabel(x):
    return all_phos_labels[x]


basePath="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main//"
train_csv_file=basePath+"/data/crop/IAM_train.csv"

valid_csv_file=basePath+"/data/crop/IAM_train1.csv"
#valid_csv_file=train_csv_file #basePath+"/data/crop/IAM_valid.csv" #args['vmap']
train_unseen_csv_file= None #args['umap']

train_folder= basePath+"/data/crop//cropYoloV3PhoscTrain//"
valid_folder=basePath+"/data/crop/IAM_valid"

BATCH_SIZE=1
EPOCHS=2

df_train=pd.read_csv(train_csv_file,nrows=10000)
df_valid=pd.read_csv(valid_csv_file,nrows=10000)

print(df_train.shape,df_valid.shape)

train_word_phos_label=gen_label(list(set(df_train['Word'])))
valid_word_phos_label=gen_label(list(set(df_valid['Word'])))
all_phos_labels={**train_word_phos_label,**valid_word_phos_label}
train_word_phoc_label=gen_phoc_label(list(set(df_train['Word'])))
valid_word_phoc_label=gen_phoc_label(list(set(df_valid['Word'])))
all_phoc_labels={**train_word_phoc_label,**valid_word_phoc_label}

type(train_word_phos_label)
len(train_word_phos_label.keys())

df_train['PhosLabel']=df_train['Word'].apply(getphoslabel)
df_valid['PhosLabel']=df_valid['Word'].apply(getphoslabel)
df_train['PhocLabel']=df_train['Word'].apply(getphoclabel)
df_valid['PhocLabel']=df_valid['Word'].apply(getphoclabel)


BATCH_SIZE=2




2022-04-30 00:16:23.876207: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


(10000, 3) (7, 3)


In [15]:


import cv2
class DataSequence(Sequence):
    def __init__(self, df, batch_size,dataType):
        self.df = df # your pandas dataframe
        self.bsz = batch_size # batch size
        self.dataType=dataType
        
        self.basePath="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main//"
        
        self.train_folder= basePath+"/data/crop//cropYoloV3PhoscTrain//"
        self.valid_folder=basePath+"/data/crop//cropYoloV3PhoscTrain//" #self.train_folder #basePath+"/data/crop/IAM_valid//"

        # Take labels and a list of image locations in memory
        self.labels=[]
        for i in range(len(self.df)):
            self.labels.append(
                {"phosnet":np.asarray(self.df['PhosLabel'].iloc[i]).astype(np.float32),
                 "phocnet":np.asarray(self.df['PhocLabel'].iloc[i]).astype(np.float32)})


        #print("\n\t labels:",self.labels)
        #print(len(self.labels))
        self.im_list = self.df['Image'].tolist()

    def __len__(self):
        # compute number of batches to yield
        return int(math.ceil(len(self.df) / float(self.bsz)))

    def on_epoch_end(self):
        # Shuffles indexes after each epoch if in training mode
        self.indexes = range(len(self.im_list))
        self.indexes = random.sample(self.indexes, k=len(self.indexes))
        
    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx * self.bsz: (idx + 1) * self.bsz])

    def get_batch_features(self, idx):
        # Fetch a batch of inputs
        #return np.array([img_to_array(load_img(im)) for im in self.im_list[idx * self.bsz: (1 + idx) * self.bsz]])
        
        if self.dataType=="train":
            im=self.im_list[idx]
            
            
            print("="*10)
            print("1.train:",os.path.isfile(self.train_folder+im))
            
            #print("\n 1.file name:",self.train_folder+im)
            
            tempImage1=load_img(self.train_folder+im)
            tempImage11=cv2.imread(self.train_folder+im)
            
            #img_to_array(load_img(self.train_folder+im))
            print("\n\t \t 1.tempImage:",tempImage1.size,"#",tempImage11.shape," \t im:",im," type:",type(tempImage11))
            print("\n\t \t 1.tempImage type:",tempImage11[0,0])      

            im=self.im_list[idx+1]
            tempImage2=load_img(self.train_folder+im)
            tempImage22=cv2.imread(self.train_folder+im)

            #print("\n 2.file name:",self.train_folder+im)

            
            print("2.train:",os.path.isfile(self.train_folder+im))
            print("\n\t\t 2.tempImage:",tempImage2.size,"#",tempImage22.shape," \t im:",im," type:",type(tempImage22))
            print("\n\t \t 2.tempImage type:",tempImage22[0,0])
            print("+"*10)

            
            """
            tempList=[]
            for im in self.im_list[idx * self.bsz: (1 + idx) * self.bsz]:
                
                tempList.append(img_to_array(load_img(self.train_folder+im)))
            """ 
            
            return np.array([img_to_array(load_img(self.train_folder+im,target_size=(50,250))) for im in self.im_list[idx * self.bsz: (1 + idx) * self.bsz]],dtype=object)
        
        elif self.dataType=="valid":
            
            return np.array([img_to_array(load_img(self.valid_folder+im,target_size=(50,250))) for im in self.im_list[idx * self.bsz: (1 + idx) * self.bsz]],dtype=object)
               
    
    def __getitem__(self, idx):
        
        
    
        batch_x = self.get_batch_features(idx)
        print("batch_x",len(batch_x))
        #batch_x =batch_x.astype(np.float32)
        #batch_x =np.asarray(batch_x).astype(np.float32)
        #np.asarray(x_train).astype(np.float32)
        batch_y = self.get_batch_labels(idx)
        l1=[]
        l2=[]
        for x in batch_y:
            l1.append(x['phosnet'])
            l2.append(x['phocnet'])
        #return batch_x, batch_y
        return batch_x,{'phosnet':np.asarray(l1),'phocnet':np.asarray(l2)}
        #return batch_x


In [16]:
train_sequence = DataSequence(df_train, BATCH_SIZE,dataType="train")
valid_sequence = DataSequence(df_valid, BATCH_SIZE,dataType="valid") 


In [13]:
# import cv2

#len(train_sequence)

i=0
expCount=0
noExp=0

for i in range(df_train.shape[0]):
    
    
    #train_sequence[i][0][0]
    
    if i<5010:
        continue

    
    #print("Image number:",i,type(train_sequence[i][0][0])," ",train_sequence[i][0].shape)
    try:
        #print(1)
        noExp+=1
        #train_sequence[i][0][0]
        print(type(train_sequence[i][0][0])," ",train_sequence[i][0].shape)
        #print(type(train_sequence[i][0])," ",train_sequence[i][0].dtype)

        #img=train_sequence[i][0][0]
        #print(train_sequence[i][0][0].dtype)
        #print("\t\t i=",i)
        
    except Exception as e:
        expCount+=1
        print(" exception:",e," -->",i,"expCount:",expCount)
        #input("check!!!!!!")
    #input(" complete!!")
print(" exp:",expCount,i)

 exp: 0 9999


In [18]:
i=0
expCount=0
noExp=0

for i in range(df_train.shape[0]):
    
    if i<5010-20:
        continue

    train_sequence[i][0][0]


1.train: True

	 	 1.tempImage: (250, 50) # (50, 250, 3)  	 im: b04-187.png_62_[315, 1597, 386, 1752]_of_1_.png  type: <class 'numpy.ndarray'>

	 	 1.tempImage type: [244 244 244]
2.train: True

		 2.tempImage: (250, 50) # (50, 250, 3)  	 im: a01-053x.png_70_[901, 2135, 987, 2214]_to_3_.png  type: <class 'numpy.ndarray'>

	 	 2.tempImage type: [240 240 240]
++++++++++
batch_x 2
1.train: True

	 	 1.tempImage: (250, 50) # (50, 250, 3)  	 im: a01-053x.png_70_[901, 2135, 987, 2214]_to_3_.png  type: <class 'numpy.ndarray'>

	 	 1.tempImage type: [240 240 240]
2.train: True

		 2.tempImage: (250, 50) # (50, 250, 3)  	 im: k04-079.png_74_[347, 1481, 489, 1602]_she_3_.png  type: <class 'numpy.ndarray'>

	 	 2.tempImage type: [251 251 251]
++++++++++
batch_x 2
1.train: True

	 	 1.tempImage: (250, 50) # (50, 250, 3)  	 im: k04-079.png_74_[347, 1481, 489, 1602]_she_3_.png  type: <class 'numpy.ndarray'>

	 	 1.tempImage type: [251 251 251]
2.train: True

		 2.tempImage: (250, 50) # (50, 250, 3) 

IndexError: index 0 is out of bounds for axis 0 with size 0

"""
    below part check phosc crops data from train dataframe and 
    only keeps the rows for which corresponding images are present 
    
"""

In [1]:
import pandas as pd
import os

In [3]:
dfPath="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/crop/IAM_train.csv"
trainCropPath="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/crop/cropYoloV3PhoscTrain//"

In [5]:
df=pd.read_csv(dfPath)
print("df.shape:",df.shape," column names:",df.columnn)

allImages=os.listdir(trainCropPath)
print(" # images:",len(allImages))

                   
                   
"""
df = pd.DataFrame()
df = df.append({'name': 'Zed', 'age': 9, 'height': 2}, ignore_index=True)

dfNew = dfNew.append({'Image': 'Zed', 'Word': 9, 'Writer': 2}, ignore_index=True)

"""

df.shape: (276363, 3)  column names: Index(['Image', 'Word', 'Writer'], dtype='object')
 # images: 267247


"\ndf = pd.DataFrame()\ndf = df.append({'name': 'Zed', 'age': 9, 'height': 2}, ignore_index=True)\n"

In [14]:

foundImages=0
dfNew=pd.DataFrame(columns=['Image', 'Word', 'Writer'])
                   
for indx,row in df.iterrows():
    
    imgName=row.Image
    
    #print("imgName:",imgName)
    
    imgPath=os.path.join(trainCropPath,imgName)
    
    #print(" is file:",os.path.isfile(imgPath))
    
    if os.path.isfile(imgPath):
        dfNew = dfNew.append({'Image': row.Image, 'Word': row.Word, 'Writer': row.Writer}, ignore_index=True)    
        foundImages+=1
    
    if indx>100:
        break

print("foundImages:",foundImages)

foundImages: 102


In [15]:
df.head(100)

,Image,Word,Writer
0,"b06-036.png_106_[481, 1976, 627, 2086]_quest_3...",quest,b06-036
1,"a04-043.png_30_[786, 1482, 957, 1523]_main_2_.png",main,a04-043
2,"m06-106.png_81_[505, 1858, 647, 1926]_flesh_2_...",flesh,m06-106
3,"n06-148.png_45_[753, 2208, 836, 2234]_as_.png",as,n06-148
4,"e04-103.png_96_[462, 908, 665, 1034]_plank_2_.png",plank,e04-103
...,...,...,...
95,"m02-066.png_82_[513, 1439, 745, 1522]_interest...",interest,m02-066
96,"d01-019.png_18_[1185, 1468, 1361, 1544]_who_1_...",who,d01-019
97,"d06-020.png_2_[825, 1312, 1287, 1391]_possibil...",possibilities,d06-020
98,"c03-007f.png_19_[359, 2315, 513, 2420]_gral_.png",gral,c03-007f


In [16]:
df.Writer.value_counts()


h04-082    362
f04-074    353
j01-066    351
f02-044    345
f01-066    344
          ... 
a03-089     69
g04-108     67
b04-208     59
m04-251     48
p02-155     31
Name: Writer, Length: 1304, dtype: int64